In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os
import sys
import pprint
pp = pprint.PrettyPrinter(indent=4)
sys.path.append("/home/muncok/DL/projects/sv_system/")

Populating the interactive namespace from numpy and matplotlib


# Dataframes

In [2]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/wav/'
data_df = pd.read_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Reddots_Dataframe.pkl')

In [3]:
### sentence duration ###

# import librosa
# audio_lengths = []
# for idx, row in data_df.sample(n=10000).iterrows():
#     file_path = os.path.join(data_dir, row.spk, row.file)
#     x, sr = librosa.load(file_path, sr=16000)
#     audio_lengths.append(len(x)/16000)

### Dataset split

In [4]:
#TODO: correct split
all_spks = list(data_df.spk.unique())
all_sents = data_df.sent.unique()
uttrs_counts = data_df.spk.value_counts()

In [5]:
si_spks = list(uttrs_counts[uttrs_counts > 100].index)
sv_spks = list(uttrs_counts[uttrs_counts <= 100].index)

### Write Manifests

In [53]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_train, 'val':si_val, 'test':si_test}
manifest_dir = "../manifests/reddots/fewshot"

for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = all_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

../manifests/voxc/fewshot/si_voxc_train_manifest.csv was written
../manifests/voxc/fewshot/si_voxc_val_manifest.csv was written


In [26]:
import random 

si_df = data_df[data_df.spk.isin(si_spks)]

# random sampling
si_random_train = si_df.sample(frac=0.8)
si_random_test = si_df.drop(index=si_random_train.index)
si_random_val = si_random_test.sample(frac=0.5)
si_random_test = si_random_test.drop(index=si_random_val.index) 
print("[random] train:{}, val:{}, test:{}".format(len(si_random_train), len(si_random_val), len(si_random_test)))

[random] train:120605, val:30151, test:30151
